# Step 1: Data and Model Preparation

In [1]:
exp_name = 'exp_052'

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

In [3]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_2_rpn import GraphBatcher

In [4]:
from src.tensorflow.models.vsp_model_002 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_02 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1234)
np.random.seed(1235)

In [6]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_2.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [7]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'split_stanford.pkl'))
misc = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'misc_stanford_2.pkl'))
img_no_ent = misc['img_no_ent']
img_no_pred = misc['img_no_pred']
image_size = misc['img_size']

In [8]:
train_idx = np.asarray(list(set(train_idx) - set(img_no_ent)))
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [9]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi', 'coords.pkl'))

In [10]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi', 'xfeat_proposals_inresnet_float32_300x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [11]:
batcher.next_batch()

{'image_id': array([2389100, 2349756, 2356381, 2346205, 2357228, 2346880, 2405272,
        2359745, 2406090, 2398375, 2373318, 2360252, 2347289, 2351812,
        2351527, 2374914, 2399717, 2414753, 2410101,    3310, 2346577,
        2407856, 2347272,  713277, 2404188, 2348400,     336, 2409023,
        2380054, 2379254, 2389715, 2364769]),
 'ent_lbl': array([[ 90, 114, 124, 139, 149,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 22,  22,  22,  23,  26,  26,  76,  76,  77, 115, 115, 124, 137,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 26,  42,  77, 145, 145, 148, 135, 134, 134,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  9,  22,  35,  35,  93, 106, 116, 132, 145, 124, 132,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  6,  24, 132,  59,  59,  76, 116, 123, 129, 132,  38,  38,   0,
           0,   0,   0,   0,   0,   0

In [12]:
batcher.key_type

int

In [39]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi', 'xfeat_proposals_inresnet_float32_300x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [40]:
tf.reset_default_graph()

In [41]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=3,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



In [42]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training

In [14]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 300,
    'filter_nonoverlap': True,
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 300,
    'filter_nonoverlap': True,
})


In [15]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [16]:
trainer.train (
    max_steps=30000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


../../src/eval/graph_eval_3.py:95: RuntimeWarning: invalid value encountered in true_divide
  per_image_pred_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))
../../src/eval/graph_eval_3.py:219: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))


Step 1000: loss = 4.120999813079834 (3603.570 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.198, 'PerImagePredMR': 0.979, 'PerImageOneHopMP_at100_iou_0.5': 0.006, 'PerImageOneHopMR_at100_iou_0.5': 0.076, 'PerImageOneHopMP_at50_iou_0.5': 0.01, 'PerImageOneHopMR_at50_iou_0.5': 0.066}
Validation Results: {'PerImagePredMP': 0.203, 'PerImagePredMR': 0.98, 'PerImageOneHopMP_at100_iou_0.5': 0.005, 'PerImageOneHopMR_at100_iou_0.5': 0.073, 'PerImageOneHopMP_at50_iou_0.5': 0.009, 'PerImageOneHopMR_at50_iou_0.5': 0.063}
Step 2000: loss = 3.9790000915527344 (7109.952 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.216, 'PerImagePredMR': 0.981, 'PerImageOneHopMP_at100_iou_0.5': 0.007, 'PerImageOneHopMR_at100_iou_0.5': 0.09, 'PerImageOneHopMP_at50_iou_0.5': 0.012, 'PerImageOneHopMR_at50_iou_0.5': 0.079}
Validation Results: {'PerImagePredMP': 0.218, 'PerImagePredMR': 0.984, 'PerImageOneHopMP_at100_iou_0.5': 0.006, 'PerImageOneHopMR_at100_iou_0.5': 0.072, 'PerImageOneHopMP_

In [17]:
trainer.train (
    max_steps=20000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 31000: loss = 3.3350000381469727 (2270.863 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.261, 'PerImagePredMR': 0.983, 'PerImageOneHopMP_at100_iou_0.5': 0.014, 'PerImageOneHopMR_at100_iou_0.5': 0.141, 'PerImageOneHopMP_at50_iou_0.5': 0.024, 'PerImageOneHopMR_at50_iou_0.5': 0.121}
Validation Results: {'PerImagePredMP': 0.263, 'PerImagePredMR': 0.987, 'PerImageOneHopMP_at100_iou_0.5': 0.013, 'PerImageOneHopMR_at100_iou_0.5': 0.122, 'PerImageOneHopMP_at50_iou_0.5': 0.021, 'PerImageOneHopMR_at50_iou_0.5': 0.108}
Step 32000: loss = 2.9519999027252197 (4497.613 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.264, 'PerImagePredMR': 0.986, 'PerImageOneHopMP_at100_iou_0.5': 0.015, 'PerImageOneHopMR_at100_iou_0.5': 0.15, 'PerImageOneHopMP_at50_iou_0.5': 0.025, 'PerImageOneHopMR_at50_iou_0.5': 0.133}
Validation Results: {'PerImagePredMP': 0.265, 'PerImagePredMR': 0.987, 'PerImageOneHopMP_at100_iou_0.5': 0.013, 'PerImageOneHopMR_at100_iou_0.5': 0.127, 'PerImageOneH

In [18]:
trainer.train (
    max_steps=5000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 51000: loss = 3.2269999980926514 (2159.273 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.268, 'PerImagePredMR': 0.988, 'PerImageOneHopMP_at100_iou_0.5': 0.015, 'PerImageOneHopMR_at100_iou_0.5': 0.158, 'PerImageOneHopMP_at50_iou_0.5': 0.025, 'PerImageOneHopMR_at50_iou_0.5': 0.136}
Validation Results: {'PerImagePredMP': 0.268, 'PerImagePredMR': 0.987, 'PerImageOneHopMP_at100_iou_0.5': 0.013, 'PerImageOneHopMR_at100_iou_0.5': 0.132, 'PerImageOneHopMP_at50_iou_0.5': 0.023, 'PerImageOneHopMR_at50_iou_0.5': 0.116}
Step 52000: loss = 2.868000030517578 (4318.722 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.266, 'PerImagePredMR': 0.99, 'PerImageOneHopMP_at100_iou_0.5': 0.015, 'PerImageOneHopMR_at100_iou_0.5': 0.156, 'PerImageOneHopMP_at50_iou_0.5': 0.025, 'PerImageOneHopMR_at50_iou_0.5': 0.136}
Validation Results: {'PerImagePredMP': 0.269, 'PerImagePredMR': 0.987, 'PerImageOneHopMP_at100_iou_0.5': 0.013, 'PerImageOneHopMR_at100_iou_0.5': 0.134, 'PerImageOneHo

In [19]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [43]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-55000'))

INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_052/ckpt-55000


In [44]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_with_duplicates.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [45]:
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 300,
    'filter_nonoverlap': True
})


In [46]:
test_idx = np.asarray(list(set(test_idx) - set(img_no_ent)))

In [47]:
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi', 'xfeat_proposals_inresnet_float32_300x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [50]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
ent_lbl shape (32, 31)
ent_lbl shape (32, 31)
ent_box shape (32, 31, 4)
pred_lbl shape (32, 19)
pred_roles shape (32, 2, 19, 31)
ent_emb shape (32, 31, 300)
pred_emb shape (32, 19, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
Batch 1/997 done in 0 seconds. Spent time for read=0.08 (avg 0.08), processing=0.25 (avg 0.25), write=0.00 (avg 0.00) seconds.
ent_lbl shape (32, 25)
ent_lbl shape (32, 25)
ent_box shape (32, 25, 4)
pred_lbl shape (32, 33)
pred_roles shape (32, 2, 33, 25)
ent_emb shape (32, 25, 300)
pred_emb shape (32, 33, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 28)
ent_lbl shape (32, 28)
ent_box shape (32, 28, 4)
pred_lbl shape (32, 24)
pred_roles shape (32, 2, 24, 28)
ent_emb shape (32, 28, 300)
pred_emb shape (32, 24, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 

ent_lbl shape (32, 21)
ent_lbl shape (32, 21)
ent_box shape (32, 21, 4)
pred_lbl shape (32, 33)
pred_roles shape (32, 2, 33, 21)
ent_emb shape (32, 21, 300)
pred_emb shape (32, 33, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 31)
ent_lbl shape (32, 31)
ent_box shape (32, 31, 4)
pred_lbl shape (32, 29)
pred_roles shape (32, 2, 29, 31)
ent_emb shape (32, 31, 300)
pred_emb shape (32, 29, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 34)
pred_roles shape (32, 2, 34, 27)
ent_emb shape (32, 27, 300)
pred_emb shape (32, 34, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 25)
p

ent_lbl shape (32, 28)
ent_lbl shape (32, 28)
ent_box shape (32, 28, 4)
pred_lbl shape (32, 28)
pred_roles shape (32, 2, 28, 28)
ent_emb shape (32, 28, 300)
pred_emb shape (32, 28, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 31)
ent_lbl shape (32, 31)
ent_box shape (32, 31, 4)
pred_lbl shape (32, 21)
pred_roles shape (32, 2, 21, 31)
ent_emb shape (32, 31, 300)
pred_emb shape (32, 21, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 31)
ent_lbl shape (32, 31)
ent_box shape (32, 31, 4)
pred_lbl shape (32, 23)
pred_roles shape (32, 2, 23, 31)
ent_emb shape (32, 31, 300)
pred_emb shape (32, 23, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 29)
ent_lbl shape (32, 29)
ent_box shape (32, 29, 4)
pred_lbl shape (32, 18)
p

ent_lbl shape (32, 24)
ent_lbl shape (32, 24)
ent_box shape (32, 24, 4)
pred_lbl shape (32, 13)
pred_roles shape (32, 2, 13, 24)
ent_emb shape (32, 24, 300)
pred_emb shape (32, 13, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 31)
ent_lbl shape (32, 31)
ent_box shape (32, 31, 4)
pred_lbl shape (32, 17)
pred_roles shape (32, 2, 17, 31)
ent_emb shape (32, 31, 300)
pred_emb shape (32, 17, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 25)
ent_lbl shape (32, 25)
ent_box shape (32, 25, 4)
pred_lbl shape (32, 21)
pred_roles shape (32, 2, 21, 25)
ent_emb shape (32, 25, 300)
pred_emb shape (32, 21, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 21)
p

ent_lbl shape (32, 39)
ent_lbl shape (32, 39)
ent_box shape (32, 39, 4)
pred_lbl shape (32, 20)
pred_roles shape (32, 2, 20, 39)
ent_emb shape (32, 39, 300)
pred_emb shape (32, 20, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 15)
pred_roles shape (32, 2, 15, 27)
ent_emb shape (32, 27, 300)
pred_emb shape (32, 15, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 24)
ent_lbl shape (32, 24)
ent_box shape (32, 24, 4)
pred_lbl shape (32, 19)
pred_roles shape (32, 2, 19, 24)
ent_emb shape (32, 24, 300)
pred_emb shape (32, 19, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 32)
ent_lbl shape (32, 32)
ent_box shape (32, 32, 4)
pred_lbl shape (32, 20)
p

...

ent_lbl shape (32, 31)
ent_lbl shape (32, 31)
ent_box shape (32, 31, 4)
pred_lbl shape (32, 18)
pred_roles shape (32, 2, 18, 31)
ent_emb shape (32, 31, 300)
pred_emb shape (32, 18, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 43)
ent_lbl shape (32, 43)
ent_box shape (32, 43, 4)
pred_lbl shape (32, 18)
pred_roles shape (32, 2, 18, 43)
ent_emb shape (32, 43, 300)
pred_emb shape (32, 18, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 33)
ent_lbl shape (32, 33)
ent_box shape (32, 33, 4)
pred_lbl shape (32, 28)
pred_roles shape (32, 2, 28, 33)
ent_emb shape (32, 33, 300)
pred_emb shape (32, 28, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 25)
ent_lbl shape (32, 25)
ent_box shape (32, 25, 4)
pred_lbl shape (32, 31)
p

ent_lbl shape (32, 26)
ent_lbl shape (32, 26)
ent_box shape (32, 26, 4)
pred_lbl shape (32, 17)
pred_roles shape (32, 2, 17, 26)
ent_emb shape (32, 26, 300)
pred_emb shape (32, 17, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 26)
ent_lbl shape (32, 26)
ent_box shape (32, 26, 4)
pred_lbl shape (32, 19)
pred_roles shape (32, 2, 19, 26)
ent_emb shape (32, 26, 300)
pred_emb shape (32, 19, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 19)
ent_lbl shape (32, 19)
ent_box shape (32, 19, 4)
pred_lbl shape (32, 18)
pred_roles shape (32, 2, 18, 19)
ent_emb shape (32, 19, 300)
pred_emb shape (32, 18, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 16)
ent_lbl shape (32, 16)
ent_box shape (32, 16, 4)
pred_lbl shape (32, 15)
p

ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 44)
pred_roles shape (32, 2, 44, 27)
ent_emb shape (32, 27, 300)
pred_emb shape (32, 44, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 15)
ent_lbl shape (32, 15)
ent_box shape (32, 15, 4)
pred_lbl shape (32, 22)
pred_roles shape (32, 2, 22, 15)
ent_emb shape (32, 15, 300)
pred_emb shape (32, 22, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 24)
ent_lbl shape (32, 24)
ent_box shape (32, 24, 4)
pred_lbl shape (32, 20)
pred_roles shape (32, 2, 20, 24)
ent_emb shape (32, 24, 300)
pred_emb shape (32, 20, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 17)
ent_lbl shape (32, 17)
ent_box shape (32, 17, 4)
pred_lbl shape (32, 18)
p

ent_lbl shape (32, 29)
ent_lbl shape (32, 29)
ent_box shape (32, 29, 4)
pred_lbl shape (32, 18)
pred_roles shape (32, 2, 18, 29)
ent_emb shape (32, 29, 300)
pred_emb shape (32, 18, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 18)
pred_roles shape (32, 2, 18, 27)
ent_emb shape (32, 27, 300)
pred_emb shape (32, 18, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 27)
ent_lbl shape (32, 27)
ent_box shape (32, 27, 4)
pred_lbl shape (32, 28)
pred_roles shape (32, 2, 28, 27)
ent_emb shape (32, 27, 300)
pred_emb shape (32, 28, 300)
num_ent shape (32,)
num_pred shape (32,)
proposal_features shape (32, 300, 1536)
proposal_boxes shape (32, 300, 4)
ent_lbl shape (32, 22)
ent_lbl shape (32, 22)
ent_box shape (32, 22, 4)
pred_lbl shape (32, 22)
p

KeyboardInterrupt: 

In [49]:
test_results = test_evaluator.evaluate()
test_results

{'PerImagePredMP': 0.2614449742753169,
 'PerImagePredMR': 0.9843629786164853,
 'PerImageOneHopMP_at100_iou_0.5': 0.013283975404693189,
 'PerImageOneHopMR_at100_iou_0.5': 0.1423840503163806,
 'PerImageOneHopMP_at50_iou_0.5': 0.021906763709373823,
 'PerImageOneHopMR_at50_iou_0.5': 0.1255273464481997}

{'PerImagePredMP': 0.2614449742753169,
 'PerImagePredMR': 0.9843629786164853,
 'PerImageOneHopMP_at100_iou_0.5': 0.013283975404693189,
 'PerImageOneHopMR_at100_iou_0.5': 0.1423840503163806,
 'PerImageOneHopMP_at50_iou_0.5': 0.021906763709373823,
 'PerImageOneHopMR_at50_iou_0.5': 0.1255273464481997}